## 📦 Import Required Libraries

This section imports all necessary Python libraries for the analysis:
- **Earth Engine (ee)**: Google's cloud platform for geospatial analysis
- **geemap**: Interactive mapping and visualization
- **numpy, scipy**: Numerical computing and statistical analysis
- **matplotlib, plotly**: Data visualization (static and interactive charts)
- **ipywidgets**: Interactive user interface components
- **gdal**: Geospatial data processing

In [1]:
# Standard library
import os
import datetime

# Third-party libraries
import ee
import json
import geemap
import requests
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.signal import savgol_filter
from osgeo import gdal

# Jupyter / interactive widgets
import ipywidgets as widgets
from ipywidgets import HTML
from ipyleaflet import WidgetControl


## 🔐 Google Earth Engine Authentication

Authenticates and initializes connection to Google Earth Engine:
1. **Authenticate**: Verifies user credentials with Google
2. **Initialize**: Connects to your GEE project for data access
3. **Error Handling**: Validates project name and handles connection errors

> **Note**: You'll need to enter your GEE project name when prompted.

In [2]:
# Start session in GEE 
ee.Authenticate()

while True:
    print("Insert your project name from GEE:")
    project_name = input().strip()
    
    if not project_name:
        print("Project name cannot be empty. Please try again.")
        continue
    
    try:
        ee.Initialize(project=project_name)
        print(f"Successfully initialized project: {project_name}")
        break
    except Exception as e:
        print(f"Error initializing project: {e}")
        print("Please try again.")

Insert your project name from GEE:
Project name cannot be empty. Please try again.
Insert your project name from GEE:
Project name cannot be empty. Please try again.
Insert your project name from GEE:
Project name cannot be empty. Please try again.
Insert your project name from GEE:
Successfully initialized project: ee-vl99956018


## 📅 Define Study Area and Time Periods

Sets up the spatial and temporal parameters for the analysis:
- **Pre-fire period**: January 1-5, 2025 (baseline conditions)
- **Post-fire period**: January 6-16, 2025 (immediate impact)
- **Study location**: Palisades Fire area (34.09°N, 118.53°W)
- **Analysis buffer**: 15 km radius around the fire origin

These parameters define what data will be retrieved from the satellite archive.

In [3]:
# Before fire date range
pre_start_date = ee.Date('2025-01-01')
pre_end_date = pre_start_date.advance(4, 'day')  # 2025-01-05

# After fire date range
pos_start_date = pre_end_date.advance(1, 'day')  # 2025-01-06
pos_end_date = pos_start_date.advance(10, 'day')  # 2025-01-16

# Coordinates of interest
latitude = 34.092615
longitude = -118.532875

# Region of interest (ROI) with a buffer of 15km
roi = ee.Geometry.Point(longitude, latitude).buffer(15000)


## 🎨 Configure Visualization Parameters

Defines color schemes and display settings for different map layers:
- **RGB visualization**: Natural color composite using visible light bands
- **NDVI palette**: Green (healthy vegetation) to Red (no vegetation)
- **NBR palette**: Specialized for detecting burned areas
- **dNBR palette**: Blue (recovery) to Red (severe burn)
- **Severity classes**: 5-level classification color scheme

Also initializes the interactive map centered on the study area.

In [4]:
# Define palette color for visualization (RGB, NDVI and NBR) and color bar settings

rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3500,  
    'gamma': 1.2
}


ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

nbr_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'black', 'red']
    
}

diff_nbr_vis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'white', 'red']
}


severity_vis = {
    'min': 0,
    'max': 4,
    'palette': ['green', 'yellow', 'orange', 'red', 'darkred']
}


#Initialize map with center and zoom
Map = geemap.Map(center=[latitude, longitude], zoom=13)


## 🛠️ Define Core Analysis Functions

Creates reusable functions for data processing:

1. **`get_all_collections()`**: Retrieves and processes Sentinel-2 imagery
   - Filters by date range, location, and cloud coverage
   - Calculates median composites (reduces noise from multiple images)
   - Computes NDVI and NBR indices

2. **`export_to_drive()`**: Exports results to Google Drive
   - Saves processed images as GeoTIFF files
   - Configurable export settings

3. **`create_labels_html()`**: Creates map labels for visualization

4. **`get_most_recent_date()`**: Finds the latest available cloud-free imagery

In [5]:
# Define functions

def get_all_collections(start_date: str, end_date: str) -> dict:

    """
    Generate median Sentinel-2 composites (reflectance, NDVI, and NBR)
    for a specified date range.

    Parameters
    ----------
    start_date : str
        Start date in 'YYYY-MM-DD' format.
    end_date : str
        End date in 'YYYY-MM-DD' format.

    Returns
    -------
    dict
        Dictionary containing:
        - 'image' : ee.Image
            Median reflectance composite.
        - 'ndvi' : ee.Image
            Median NDVI image.
        - 'nbr' : ee.Image
            Median NBR image.
    """

    try:
        # Selecting Sentinel-2 collection with defined date range, ROI and cloud cover filter
        collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(roi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

        # Generating median reflectance composite
        image = collection.select('B8', 'B12', 'B4', 'B3', 'B2').median().clip(roi)

        # Generating median NDVI composite
        ndvi = collection.select(['B8', 'B4']) \
            .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
            .median().clip(roi)
        
        # Generating median NBR composite
        nbr = collection.select(['B8', 'B12']) \
            .map(lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR')) \
            .median().clip(roi)

        return {'image': image, 'ndvi': ndvi, 'nbr': nbr}

    except Exception as e:
        print(f"Error: {e}")
        

# Export function to Google Drive
def export_to_drive(image: ee.Image, image_title: str, folder_name='GEE_exports'):

    """
    Export an Earth Engine image to Google Drive.

    Parameters
    ----------
    image : ee.Image
        Image to export.
    image_title : str
        Title for the exported image.
    folder_name : str, optional
        Name of the Google Drive folder where the image will be saved.
    """

    try:
        # Initializing export task to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=image_title,
            folder=folder_name,
            fileNamePrefix=image_title.lower().replace(' ', '_'),
            scale=10,
            region=roi,
            fileFormat='GeoTIFF',
            maxPixels=1e10
        )

        task.start()
        
        print(
            f'Started export task for {image_title}. '
            f'Check your Google Drive folder "{folder_name}" when complete.'
        )

    except Exception as e:
        print(f"Error: {e}")


# Create HTML labels for map output
def create_labels_html(legend_before: str, legend_after: str) -> tuple:

    """
    Create HTML label elements for map display.

    Parameters
    ----------
    legend_before : str
        Text for the label positioned in the top-left corner.
    legend_after : str
        Text for the label positioned in the top-right corner.

    Returns
    -------
    tuple
        Tuple of IPython.display.HTML objects:
        (before_label, after_label), styled for use in folium/geemap maps.
    """

    before_html = HTML(
        f"""<div style="position: absolute; top: 10px; left: 10px; z-index: 1000;
        background-color: white; padding: 5px; border-radius: 5px;
        font-weight: bold; color: black;">{legend_before}</div>"""
    )

    after_html = HTML(
        f"""<div style="position: absolute; top: 70px; right: 10px; z-index: 1000;
        background-color: white; padding: 5px; border-radius: 5px;
        font-weight: bold; color: black;">{legend_after}</div>"""
    )

    return before_html, after_html


# Get the most recent date of available images
def get_most_recent_date(start_date: str) -> str:

    """
    Retrieve the most recent available Sentinel-2 acquisition date
    after a given start date.

    Parameters
    ----------
    start_date : str
        Start date in 'YYYY-MM-DD' format.

    Returns
    -------
    str
        Most recent date in 'YYYY-MM-DD' format.
    """
    
    try:
        # Gets today's date
        today = datetime.date.today().isoformat()


        # Selecting Sentinel-2 collection with defined date range, ROI and cloud cover filter
        collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterDate(start_date, today) \
            .filterBounds(roi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
            .sort('system:time_start', False)

        # Getting the most recent image from the collection
        most_recent_image = collection.first()

        # Getting the date of the most recent image
        most_recent_date = ee.Date(
            most_recent_image.get('system:time_start')
        ).format('YYYY-MM-dd')

        return most_recent_date.getInfo()

    except Exception as e:
        print(f"Error in get_most_recent_date: {e}")


## 🖼️ Generate Before/After RGB Comparison

Creates an interactive split-screen map showing fire impact:

**Data Processing:**
- Retrieves pre-fire imagery (Jan 1-5)
- Retrieves post-fire imagery (Jan 6-16)
- Creates median composites from available images

**Visualization:**
- Left side: Before fire conditions
- Right side: After fire conditions
- Layers can be toggled on/off in the map controls

**Optional**: Export images to Google Drive (commented out by default)

In [6]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN RGB

map_rgb = Map

# Pre and post fire images in RGB
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['image']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']


# Add split map to compare before and after fire images in RGB
left_layer = geemap.ee_tile_layer(pre_image, rgb_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')

map_rgb.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_rgb.addLayer(pre_image, rgb_vis, 'Before Fire', False)
map_rgb.addLayer(pos_image, rgb_vis, 'After Fire', False)


# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_rgb.add_control(WidgetControl(widget=before_html, position='topleft'))
map_rgb.add_control(WidgetControl(widget=after_html, position='topright'))

map_rgb



#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================

# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")




Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## 🌿 NDVI Analysis: Vegetation Health Comparison

Analyzes vegetation health changes using NDVI (Normalized Difference Vegetation Index):

**What is NDVI?**
- Formula: `(NIR - Red) / (NIR + Red)`
- Range: -1 to +1
- Interpretation: Higher values = healthier, denser vegetation

**Analysis:**
- Compares pre-fire vs post-fire NDVI values
- Green areas = healthy vegetation
- Red/yellow areas = stressed or dead vegetation
- Shows clear decline in vegetation health after the fire

In [7]:
def download_gee_tiff(image, filename="output.tif", scale=30):

    """
    Download a GeoTIFF image from Google Earth Engine.

    Parameters
    ----------
    image : ee.Image
        Earth Engine image to download.
    filename : str, optional
        Name of the output TIFF file.
    scale : int, optional
        Spatial resolution in meters.

    Returns
    -------
    str
        Path to the downloaded TIFF file.
    """

    try:
        # Creates directory if it doesn't exist
        os.makedirs("results", exist_ok=True)
        filename = os.path.join("results", filename)

        # Generates download URL for the image
        url = image.getDownloadURL({
            'scale': scale,
            'format': 'GEO_TIFF'
        })

        print(f"Downloading {filename} ...")
        r = requests.get(url, stream=True)
        r.raise_for_status()  # Raise an exception for bad status codes

        # Saves the image to a file with limited memory usage
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        print(f"Saved TIFF: {filename}")
        return filename
    
    except Exception as e:
        print(f"Unexpected error in download_gee_tiff: {e}")


def tif_to_pdf(tif_path, pdf_path, gamma=1.3, do_percentile_stretch=True):

    """
    Convert a TIFF image to a PDF with optional percentile stretching
    and gamma correction.

    Parameters
    ----------
    tif_path : str
        Path to the input TIFF file.
    pdf_path : str
        Path to the output PDF file.
    gamma : float, optional
        Gamma correction factor.
    do_percentile_stretch : bool, optional
        If True, apply a 2–98 percentile stretch before normalization.

    Returns
    -------
    None
    """
    
    try:
        # Creates directory if it doesn't exist
        os.makedirs("results", exist_ok=True)

        tif_path = os.path.join("results", os.path.basename(tif_path))
        pdf_path = os.path.join("results", os.path.basename(pdf_path))

        # Reads TIFF file using GDAL
        ds = gdal.Open(tif_path)
        if ds is None:
            raise FileNotFoundError(f"Could not open TIFF file: {tif_path}")
        
        array = ds.ReadAsArray()

        if array.ndim == 3:
            array = array.transpose((1, 2, 0))
        
        array = array.astype("float32")

        if do_percentile_stretch:
            p2 = np.percentile(array, 2)
            p98 = np.percentile(array, 98)
            array = np.clip((array - p2) / (p98 - p2), 0, 1)
        else:
            arr_min = array.min()
            arr_max = array.max()
            array = (array - arr_min) / (arr_max - arr_min)
            array = np.clip(array, 0, 1)

        array = np.power(array, 1.4 / gamma)

        plt.figure(figsize=(10, 10))
        plt.imshow(array)
        plt.axis("off")
        plt.savefig(pdf_path, bbox_inches='tight', pad_inches=0, dpi=300)
        plt.close()

        print(f"Saved PDF (gamma={gamma}): {pdf_path}")
    
    except Exception as e:
        print(f"Unexpected error in tif_to_pdf: {e}")

## 🔥 NBR Analysis: Burn Detection

Uses NBR (Normalized Burn Ratio) to identify burned areas:

**What is NBR?**
- Formula: `(NIR - SWIR) / (NIR + SWIR)`
- Specifically designed to detect burned vegetation
- More sensitive to fire damage than NDVI

**Visualization:**
- Dark areas in post-fire image = severely burned
- Comparison shows extent of fire damage
- NBR is foundation for severity classification

In [8]:
# Burn Severity Analysis functions

def classify_burn_severity(dnbr_image):
    """
    Classify burn severity based on dNBR values using USGS/BAER standards.
    
    Parameters
    ----------
    dnbr_image : ee.Image
        Differenced NBR image (pre-NBR minus post-NBR)
    
    Returns
    -------
    ee.Image
        Classified severity image with values 0-4
    """
    
    try:
        # Define severity thresholds based on USGS Fire Effects Monitoring standards
        severity = ee.Image(0) \
            .where(dnbr_image.gte(0.1), 1) \
            .where(dnbr_image.gte(0.27), 2) \
            .where(dnbr_image.gte(0.44), 3) \
            .where(dnbr_image.gte(0.66), 4) \
            .rename('severity')
        
        return severity
    except Exception as e:
        print(f"Error classifying burn severity: {e}")
        raise


def get_severity_statistics(severity_image, roi, scale=30):
    """
    Calculate area statistics for each burn severity class.
    
    Parameters
    ----------
    severity_image : ee.Image
        Classified severity image
    roi : ee.Geometry
        Region of interest
    scale : int
        Analysis scale in meters
    
    Returns
    -------
    dict
        Statistics dictionary with areas in hectares and percentages
    """
    
    try:
        severity_names = {
            0: 'Unburned',
            1: 'Low Severity',
            2: 'Moderate-Low',
            3: 'Moderate-High',
            4: 'High Severity'
        }
        
        # Calculate pixel area
        pixel_area = ee.Image.pixelArea()
        
        # Get total area
        total_area_m2 = pixel_area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=roi,
            scale=scale,
            maxPixels=1e13
        ).get('area').getInfo()
        total_area_ha = total_area_m2 / 10000
        
        stats = {'total_area_hectares': total_area_ha}
        
        # Calculate area for each severity class
        for severity_value, severity_name in severity_names.items():
            # Create mask for this severity class
            mask = severity_image.eq(severity_value)
            area_image = pixel_area.updateMask(mask)
            
            # Calculate area
            area_m2 = area_image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=roi,
                scale=scale,
                maxPixels=1e13
            ).get('area')
            
            # Handle None values
            area_m2 = ee.Number(area_m2).getInfo() if area_m2 else 0
            area_ha = area_m2 / 10000
            percentage = (area_ha / total_area_ha) * 100 if total_area_ha > 0 else 0
            
            stats[severity_name] = {
                'hectares': round(area_ha, 2),
                'acres': round(area_ha * 2.471, 2),
                'percentage': round(percentage, 2)
            }
        
        return stats
    except Exception as e:
        print(f"Error calculating severity statistics: {e}")
        raise


def print_severity_report(stats):
    """
    Print a formatted burn severity report.
    """
    try:
        print("\n" + "="*60)
        print("BURN SEVERITY ANALYSIS REPORT")
        print("="*60)
        print(f"\nTotal Analysis Area: {stats['total_area_hectares']:.2f} hectares")
        print(f"                     ({stats['total_area_hectares']*2.471:.2f} acres)")
        print("\n" + "-"*60)
        print(f"{'Severity Class':<20} {'Hectares':<12} {'Acres':<12} {'%':<8}")
        print("-"*60)
        
        for severity_name in ['Unburned', 'Low Severity', 'Moderate-Low', 'Moderate-High', 'High Severity']:
            data = stats[severity_name]
            print(f"{severity_name:<20} {data['hectares']:<12.2f} {data['acres']:<12.2f} {data['percentage']:<8.2f}")
        
        print("="*60 + "\n")
    except Exception as e:
        print(f"Error printing severity report: {e}")
        raise


def create_severity_chart(stats):
    """
    Create an interactive pie chart of burn severity distribution.
    """
    try:
        severity_order = ['Unburned', 'Low Severity', 'Moderate-Low', 'Moderate-High', 'High Severity']
        colors = ['#2ECC71', '#F4D03F', '#E67E22', '#E74C3C', '#943126']
        
        values = [stats[name]['hectares'] for name in severity_order]

        fig = go.Figure(data=[go.Pie(
            labels=severity_order,
            values=values,
            marker=dict(colors=colors),
            textinfo='label+percent',
            hovertemplate='<b>%{label}</b><br>' +
                          'Area: %{value:.2f} ha<br>' +
                          'Percentage: %{percent}<br>' +
                          '<extra></extra>'
        )])
        
        fig.update_layout(
            title='Burn Severity Distribution',
            template='plotly_white',
            showlegend=True
        )
        
        # Save chart
        os.makedirs("results", exist_ok=True)
        fig.write_html("results/severity_distribution.html")
        fig.write_image("results/severity_distribution.pdf")
        
        print("Severity chart saved to 'results/severity_distribution.html'")
        fig.show(renderer="browser")
    except Exception as e:
        print(f"Error creating severity chart: {e}")
        raise

## 📊 Fire Severity Classification (dNBR)

Calculates fire severity using dNBR (differenced NBR):

**Methodology:**
1. Calculate dNBR = NBR_prefire - NBR_postfire
2. Classify into 5 severity levels:
   - Unburned: dNBR < 0.1
   - Low: 0.1 - 0.27
   - Moderate-Low: 0.27 - 0.44
   - Moderate-High: 0.44 - 0.66
   - High: ≥ 0.66

**Output:**
- Color-coded severity map
- Statistical breakdown by severity class
- Quantifies total affected area

In [9]:
# Select only RGB bands from Sentinel-2 images
pre_rgb = pre_image.select(['B4', 'B3', 'B2'])
pos_rgb = pos_image.select(['B4', 'B3', 'B2'])


# Download TIFF images 
pre_tif = download_gee_tiff(pre_rgb, "before_fire.tif", scale=30)
pos_tif = download_gee_tiff(pos_rgb, "after_fire.tif", scale=30)


# Converts TIFF to PDF
tif_to_pdf("before_fire.tif", "before_fire.pdf")
tif_to_pdf("after_fire.tif", "after_fire.pdf")


Saved TIFF: results/before_fire.tif
Saved TIFF: results/after_fire.tif
Saved PDF (gamma=1.3): results/before_fire.pdf
Saved PDF (gamma=1.3): results/after_fire.pdf


## 📈 Calculate Area Statistics by Severity

Quantifies the impact by computing area for each severity class:

**Process:**
1. Convert classified image to vector features
2. Calculate area in square kilometers for each class
3. Sum total affected area
4. Display results as structured data

**Business Value:**
- Provides concrete numbers for reporting
- Supports insurance claims and damage assessment
- Enables comparison with other fire events

In [10]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NDVI

map_ndvi = Map


# Pre and post fire images in NDVI
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['ndvi']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['ndvi']


# Add split map to compare before and after fire images in NDVI
left_layer = geemap.ee_tile_layer(pre_image, ndvi_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, ndvi_vis, 'After Fire')

map_ndvi.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_ndvi.addLayer(pre_image, ndvi_vis, 'Before Fire', False)
map_ndvi.addLayer(pos_image, ndvi_vis, 'After Fire', False)


# NDVI color bar
map_ndvi.add_colorbar(
    vis_params=ndvi_vis,
    label='NDVI',
    layer_name='NDVI',
    orientation='vertical',  
    position='bottomright'
    )

# Add HTML labels to the map
before_html, adter_html = create_labels_html('Before', 'After')
map_ndvi.add_control(WidgetControl(widget=before_html, position='topleft'))
map_ndvi.add_control(WidgetControl(widget=adter_html, position='topright'))

map_ndvi


#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================


# export_to_drive(pre_image, "before_fire_NDVI_image")
# export_to_drive(pos_image, "after_fire_NDVI_image")


Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## 💾 Download NDVI/NBR Time-Series Data

Retrieves historical vegetation index data for trend analysis:

**Function: `download_ndvi_nbr_data()`**
- Fetches Sentinel-2 images from specified date range
- Filters by cloud coverage (<10%)
- Extracts mean NDVI and NBR values for the study area
- Returns data as a structured dictionary with dates and values

**Purpose:** Provides raw data for creating time-series visualizations

In [11]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NBR

map_nbr = Map

# Pre and post fire images in NBR
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['nbr']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['nbr']


# Calculate difference between pre and post fire images in NBR
dnbr = pre_image.subtract(pos_image)

# Define a threshold to identify burned areas
burned_mask = dnbr.gt(0.3)

# Calculate total burned area (in hec)
pixel_area = burned_mask.multiply(ee.Image.pixelArea()).divide(10000)  # m^2 to hec
area_burned = pixel_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=30,
    maxPixels=1e13
)


# Add split map to compare before and after fire images in NBR
left_layer = geemap.ee_tile_layer(pre_image, nbr_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, nbr_vis, 'After Fire')


map_nbr.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_nbr.addLayer(pre_image, nbr_vis, 'Before Fire', False)
map_nbr.addLayer(pos_image, nbr_vis, 'After Fire', False)


# NBR color bar
map_nbr.add_colorbar(
    vis_params=nbr_vis,
    label='NBR',
    layer_name='NBR',
    orientation='vertical',
    position='bottomleft'
)

# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_nbr.add_control(WidgetControl(widget=before_html, position='topleft'))
map_nbr.add_control(WidgetControl(widget=after_html, position='topright'))



# Map for difference in NBR
Map_diff = geemap.Map(center=[latitude, longitude], zoom=13)
Map_diff.addLayer(dnbr, diff_nbr_vis, 'ΔNBR')

display(widgets.VBox([Map, Map_diff]))



#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================


# export_to_drive(pre_image, "before_fire_NBR_image")
# export_to_drive(pos_image, "after_fire_NBR_image")
# export_to_drive(dnbr, "Difference NBR")




## 📉 Create Interactive Time-Series Visualization

Generates publication-quality plots of vegetation recovery:

**Function: `ndvi_nbr_plot()`**
1. Downloads NDVI and NBR data over 6-month period
2. Applies Savitzky-Golay smoothing filter (reduces noise)
3. Creates dual-axis interactive plot with Plotly
4. Marks fire event date with vertical line
5. Exports as HTML (interactive) and PDF (static)

**Insights:**
- Shows vegetation decline after fire
- Tracks recovery progress over time
- Identifies seasonal patterns

In [12]:
# Burn Severity Analysis

print("Running burn severity analysis...\n")

# Calculate dNBR (pre-NBR minus post-NBR)
pre_nbr = pre_image.select('NBR')
post_nbr = pos_image.select('NBR')
dnbr = pre_nbr.subtract(post_nbr).rename('dNBR')

# Classify severity
severity = classify_burn_severity(dnbr)

# Get statistics
severity_stats = get_severity_statistics(
    severity, 
    roi
)

# Print report
print_severity_report(severity_stats)

# Save statistics to JSON
with open('results/severity_statistics.json', 'w') as f:
    json.dump(severity_stats, f, indent=2)
print("Statistics saved to 'results/severity_statistics.json'\n")

# Create visualization chart
create_severity_chart(severity_stats)

# Create a new map for severity visualization
Map_severity = geemap.Map(
    center=[latitude, longitude]
)

# Add dNBR and severity layers
Map_severity.addLayer(dnbr, diff_nbr_vis, 'dNBR (continuous)', True)
Map_severity.addLayer(severity, severity_vis, 'Burn Severity (classified)', True)

# Add legend
legend_dict = {
    'Unburned': '#2ECC71',
    'Low Severity': '#F4D03F',
    'Moderate-Low': '#E67E22',
    'Moderate-High': '#E74C3C',
    'High Severity': '#943126'
}
Map_severity.add_legend(title="Burn Severity", legend_dict=legend_dict)


Running burn severity analysis...


BURN SEVERITY ANALYSIS REPORT

Total Analysis Area: 69822.28 hectares
                     (172530.85 acres)

------------------------------------------------------------
Severity Class       Hectares     Acres        %       
------------------------------------------------------------
Unburned             51543.32     127363.55    73.82   
Low Severity         7908.65      19542.28     11.33   
Moderate-Low         3834.30      9474.55      5.49    
Moderate-High        3844.39      9499.49      5.51    
High Severity        2691.62      6650.98      3.85    

Statistics saved to 'results/severity_statistics.json'

Severity chart saved to 'results/severity_distribution.html'
Opening in existing browser session.


## 🎯 Execute Time-Series Analysis

Runs the plotting function for the specified date range:

**Parameters:**
- Start date: October 1, 2024 (3 months before fire)
- End date: April 1, 2025 (3 months after fire)
- Total: 6-month analysis window

**Output:**
- Interactive HTML chart (can zoom, pan, hover for details)
- PDF export for reports and presentations
- Files saved in `results/` directory

In [13]:
def add_trace(fig, x: list, y: list, name: str, mode='lines+markers',
               line_color='blue', line_width = 2,
                 marker_color='red', marker_size = 8) -> None:
    
    """
    Add a trace to a Plotly figure.

    Parameters
    ----------
    fig : plotly.graph_objs._figure.Figure
        Figure to which the trace will be added.
    x : list
        X-values of the trace.
    y : list
        Y-values of the trace.
    name : str
        Name of the trace.
    mode : str, optional
        Plotly trace mode. Default is 'lines+markers'.
    line_color : str, optional
        Line color. Default is 'blue'.
    line_width : int, optional
        Line width. Default is 2.
    marker_color : str, optional
        Marker color. Default is 'red'.
    marker_size : int, optional
        Marker size. Default is 8.

    Returns
    -------
    None
    """
    
    try:
        # Creates a trace
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode=mode,
            name=name,
            line=dict(color=line_color, width=line_width),
            marker=dict(size=marker_size, color=marker_color),
        ))
    except Exception as e:
        print(f"Error adding trace for {name}: {e}")

def get_ndvi(image):

    """
    Add an NDVI band to an Earth Engine image.

    NDVI is computed as (B8 - B4) / (B8 + B4).

    Parameters
    ----------
    image : ee.Image
        Input Earth Engine image.

    Returns
    -------
    ee.Image
        Image with an added NDVI band.
    """

    # Computes NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def get_nbr(image):

    """
    Add an NBR band to an Earth Engine image.

    NBR is computed as (B12 - B8) / (B12 + B8).

    Parameters
    ----------
    image : ee.Image
        Input Earth Engine image.

    Returns
    -------
    ee.Image
        Image with an added NBR band.
    """
    # Computes NBR
    nbr = image.normalizedDifference(['B12', 'B8']).rename('NBR')
    return image.addBands(nbr)

def ndvi_nbr_plot(start_date:str, end_date:str) -> None:
    
    """
    Plot NDVI and NBR time series for the ROI over a given date range.

    The function computes mean NDVI and NBR values from Sentinel-2 imagery,
    applies Savitzky–Golay smoothing, and saves the result as an interactive
    HTML file.

    Parameters
    ----------
    start_date : str
        Start date in 'YYYY-MM-DD' format.
    end_date : str
        End date in 'YYYY-MM-DD' format.

    Returns
    -------
    None
    """

    try:
        # Load Sentinel-2 SR collection
        collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                        .filterDate(start_date, end_date)
                        .filterBounds(roi)
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
                        .map(get_ndvi)
                        .map(get_nbr)
                        .select(['NDVI', 'NBR']))
        

        # Extract NDVI and NBR means and date as Feature
        def reduce_img(img):
            stats = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=roi,
                scale=10,
                maxPixels=1e9
            )
            return ee.Feature(None, {
                'date': img.date().format('YYYY-MM-dd'),
                'NDVI': stats.get('NDVI'),
                'NBR': stats.get('NBR')
            })

        # Computes mean NDVI and NBR values for each image
        features = collection.map(reduce_img)

        # Extracts date, NDVI, and NBR values
        stats_list = features.aggregate_array('date').getInfo()
        ndvi_values = features.aggregate_array('NDVI').getInfo()
        nbr_values = features.aggregate_array('NBR').getInfo()

        # Arguments to smooth curves
        window = 19     
        poly = 6       

        # Smooth curves
        ndvi_smooth = savgol_filter(ndvi_values, window_length=window, polyorder=poly)
        nbr_smooth  = savgol_filter(nbr_values,  window_length=window, polyorder=poly)


        # Convert to datetime
        dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in stats_list]

        # Plot NDVI and NBR curves in the same figure
        fig = go.Figure()

        add_trace(fig, dates, ndvi_values, 'NDVI', line_color='green', marker_color='green')
        add_trace(fig, dates, ndvi_smooth, 'NDVI (smoothed)', line_color='green', marker_color='green', mode = 'lines')
        
        add_trace(fig, dates, nbr_values, 'NBR', line_color='red', marker_color='red')
        add_trace(fig, dates, nbr_smooth, 'NBR (smoothed)', line_color='red', marker_color='red', mode= "lines")


        # Add vertical line at 2025-01-07
        fig.add_vline(
            x=datetime.datetime(2025, 1, 7).timestamp() * 1000,  # Convert to milliseconds since epoch
            line_width=2,
            line_dash="dash",
            line_color="blue",
            annotation_text="First Fire detected",
            annotation_position="top right"
        )

        # Configures layout
        fig.update_layout(
            title='NDVI and NBR TimeSeries',
            xaxis_title='Date',
            yaxis_title='Index Value',
            xaxis=dict(showgrid=True),
            yaxis=dict(showgrid=True, title='Index Value'),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            template='plotly_white',
            hovermode='x unified'
        )

        # Save plot
        os.makedirs("results", exist_ok=True)
        fig.write_html("results/ndvi_nbr_plot.html")
        fig.write_image("results/ndvi_nbr_plot.pdf", width = 1400, height = 600)
        
        print("Graphic saved at 'results/ndvi_nbr_plot.html'")
        fig.show(renderer = "browser")

    except Exception as e:
        print(f"Error in ndvi_nbr_plot: {e}")


## 🔄 Compare Post-Fire to Current Conditions

Creates a final comparison showing recovery progress:

**Analysis:**
1. Retrieves post-fire imagery (Jan 6-16, 2025)
2. Finds most recent cloud-free imagery (automatically detected)
3. Creates split-screen comparison

**Purpose:**
- Track vegetation recovery over time
- Monitor restoration efforts
- Update stakeholders on current conditions

**Note:** The "most recent date" varies based on satellite coverage and cloud conditions

In [14]:
curve_start_date = '2024-10-01'
curve_end_date = '2025-04-01'

# Run the function
ndvi_nbr_plot(curve_start_date, curve_end_date)

Graphic saved at 'results/ndvi_nbr_plot.html'
Opening in existing browser session.


In [15]:
# GET IMAGES AFTER FIRE AND TODAY'S DATE

# After images
pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']

# Get most recent date
recent_date = get_most_recent_date('2025-04-01')
today = datetime.date.today().isoformat()

# Today images
today_result = get_all_collections(recent_date, today)
today_image = today_result['image']

print(f"The most recent date is {recent_date} for a cloud coverage of 10% or less.")

# Split map
left_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')
right_layer = geemap.ee_tile_layer(today_image, rgb_vis, 'Today')

Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Add layers  
Map.addLayer(pos_image, rgb_vis, 'After Fire', False)
Map.addLayer(today_image, rgb_vis, 'Today', False)



#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================


# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")


# Add map control
antes_html, depois_html = create_labels_html('After', 'Today')
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))

Map

The most recent date is 2026-02-06 for a cloud coverage of 10% or less.


Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…